# <img align="left" src="assets/panda.jpeg" width="100" height="100" style="border-radius: 50%; margin-right: 20px">  
# Cobrança em massa de pedidos sem agenda por fornecedor 

> Essa **automação** envia e-mails de cobrança de agendamentos por fornecedor para cada pedido que constar sem agenda na carteira de pedidos.

>Dica:
>
>Para executar esse código na totalidade, clique em **RUN** e depois em **RUN ALL CELLS**.

In [1]:
# bibliotecas utilizadas no projeto
import win32com.client as win32 
import pandas as pd
from datetime import datetime

In [14]:
base = pd.read_excel('planilhas/BASE_DASHBOARD_PEDIDOS_COMPRA.xlsx') # base da carteira de pedidos por fornecedor.
emails_forn = pd.read_excel('planilhas/emails_forn.xlsx') # base de contatos de emails fornecedores.
emails_amigao = pd.read_excel('planilhas/emails_amigao.xlsx')  # base de contatos internos por depto.

In [3]:
# exclusão de colunas excedentes da carteira de pedidos
base = base.drop("Unnamed: 0", axis=1)
base = base.drop("Unnamed: 22", axis=1)
base = base.drop("Unnamed: 23", axis=1)
base = base.drop("Unnamed: 24", axis=1)

In [4]:
# variavel que pega a data atual
data_atual = datetime.now().date()

In [5]:
# filtro de fornecedores com data de entrega maior ou igual ao dial atual
base_dashboard = base[base['DT_ENTREGA'].dt.date >= data_atual]

In [6]:
# filtro de fornecedores sem agendamento
fornecedores_sem_agendamento = base_dashboard[base_dashboard['DT_AGENDA'].str.startswith('SEM')]

In [7]:
# filtro de pedidos do tipo original e forma de aquisição CD
fornecedores_cd_original = fornecedores_sem_agendamento[(fornecedores_sem_agendamento['Classificacao do pedido'] == 'Original') & 
                            (fornecedores_sem_agendamento['LOCAL_ENT'] == 'CD')]

In [8]:
fornecedores_validos = fornecedores_cd_original.drop_duplicates(subset="PEDIDO") # remoção de nº de pedido duplicado

In [ ]:
# função de envio e-mail
def send_email(to, cc, subject, body):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0) # inicializa o outlook 
    
    # remove duplicatas dos e-mails
    to_unique = list(set(to))
    cc_unique = list(set(cc))
    
    # adiciona destinatários
    mail.To = ';'.join(to_unique)
    mail.CC = ';'.join(cc_unique)
    
    # adiciona assunto e corpo do e-mail
    mail.Subject = subject
    mail.Body = body
    
    try:
        # tratamento de erro no envio de emails
        mail.Send()
        return True  # indicador de e-mail enviado com sucesso
    except Exception as e:
        print(f"Erro ao enviar e-mail: {str(e)}")
        return False  # indicador de erro do envio de e-mail

# contador de emails 
emails_enviados = 0
emails_nao_enviados = 0

# agrupamento de fornecedores
for (usuario, depto, nome_fornecedor), grupo_pedidos in fornecedores_validos.groupby(['NOME USUARIO', 'DEP', 'DESC_FORN']):
    # obter informações do fornecedor e pedidos
    email_to = emails_forn.loc[emails_forn['nome_fornecedor'] == nome_fornecedor, 'email'].tolist()
    email_cc = emails_forn.loc[emails_forn['nome_fornecedor'] == nome_fornecedor, 'email_forn_cc'].tolist()

    # evita duplicação de e-mails
    email_to = list(set(email_to))
    email_cc = list(set(email_cc))

    # departamentos para cópia de e-mail
    depto_fornecedor = grupo_pedidos.iloc[0]['DEP']
    if depto_fornecedor in emails_amigao['depto'].values:
        contatos_amigao = emails_amigao.loc[emails_amigao['depto'] == depto_fornecedor, 'contatos_amigao'].tolist()
        email_cc += contatos_amigao

        # evita duplicação de e-mails
        email_cc = list(set(email_cc))

    # agrupamento de pedidos por departamento e usuário
    pedidos_usuario = grupo_pedidos.groupby(['DEP', 'NOME USUARIO', 'DESC_FORN', 'PEDIDO'])

    # corpo do e-mail
    corpo_email = f"""Prezado Fornecedor:

Identificamos em nosso sistema o(s) pedido(s) de compra pendente(s) de agendamento com elevado nível de ruptura em nossas unidades e CD’s.

FORNECEDOR: {nome_fornecedor}
COD_FORN: {grupo_pedidos.iloc[0]['COD_FORN']}
DEPARTAMENTO: {depto_fornecedor}
COMPRADOR: {usuario}

PEDIDOS:\n"""
    for (_, pedido) in pedidos_usuario:
        lead_time = pedido.iloc[0]['DT_ENTREGA'].strftime('%d/%m/%y')  # formatar datas para dd/mm/yy
        data_emissao = pedido.iloc[0]['DT_EMISSAO'].strftime('%d/%m/%y')  # formatar datas para dd/mm/yy
        # estrutura do pedido
        corpo_email += f"N° {pedido.iloc[0]['PEDIDO']} | EMITIDO EM: {data_emissao} | AGENDAR ATÉ: {lead_time}\n" 

    # adiciona departamento e comprador
    corpo_email += """


Gentileza agendar na data mais próxima disponível para mitigarmos a ruptura atual. Qualquer dificuldade sinalizar através do e-mail: agendamento@grupoamigao.com

Caso o pedido já esteja agendado desconsiderar.

No aguardo.
"""

    # enviar os e-mails
    if send_email(email_to, email_cc, 'URGENTE – PEDIDO SEM AGENDAMENTO NO PORTAL', corpo_email):
        # incremento do contador de e-mails enviados se o e-mail for enviado com sucesso
        emails_enviados += 1
        print(f'E-mail enviado para {nome_fornecedor} ({usuario}, {depto})')
    else:
        # incremento do contador de e-mails não enviados se ocorrer um erro ao enviar o e-mail
        emails_nao_enviados += 1

# print dos contadores
print(f'Total de e-mails enviados: {emails_enviados}')
print(f'Total de e-mails não enviados: {emails_nao_enviados}')

In [10]:
# Relatório para validar emails enviados por usuários, departamentos e qtd de pedidos.
#resumo_pedidos = fornecedores_validos.groupby(['DEP', 'NOME USUARIO'])['PEDIDO'].count().reset_index()

In [12]:
#caminho = "planilhas/base_depto.xlsx"

In [13]:
#resumo_pedidos.to_excel(caminho, index=False)